# Time Series Analysis with OPERA RTC-S1

### Franz J Meyer; University of Alaska Fairbanks & Josef Kellndorfer, [Earth Big Data, LLC](http://earthbigdata.com/)

This notebook will introduce you to the analysis of deep multi-temporal SAR image data stacks in the framework of *Jupyter Notebooks*. The Jupyter Notebook environment is easy to launch in any web browser for interactive data exploration with provided or new training data. Notebooks are comprised of text written in a combination of executable python code and markdown formatting including latex style mathematical equations. Another advantage of Jupyter Notebooks is that they can easily be expanded, changed, and shared with new data sets or newly available time series steps. Therefore, they provide an excellent basis for collaborative and repeatable data analysis.

**We introduce the following data analysis concepts:**

- How to load your own SAR data into Jupyter Notebooks and create a time series stack 
- How to apply calibration constants to covert initial digital number (DN) data into calibrated radar cross section information.
- How to subset images and create a time series of your subset data.
- How to explore the time-series information in SAR data stacks for environmental analysis.

---
## 0. Importing Relevant Python Packages

In this notebook we will use the following scientific libraries:

- [Pandas](https://pandas.pydata.org/) is a Python library that provides high-level data structures and a vast variety of tools for analysis. The great feature of this package is the ability to translate rather complex operations with data into one or two commands. Pandas contains many built-in methods for filtering and combining data, as well as the time-series functionality.
- [GDAL](https://www.gdal.org/) is a software library for reading and writing raster and vector geospatial data formats. It includes a collection of programs tailored for geospatial data processing. Most modern GIS systems (such as ArcGIS or QGIS) use GDAL in the background.
- [NumPy](http://www.numpy.org/) is one of the principal packages for scientific applications of Python. It is intended for processing large multidimensional arrays and matrices, and an extensive collection of high-level mathematical functions and implemented methods makes it possible to perform various operations with these objects.
- [XArray](https://docs.xarray.dev/en/stable/index.html) provides a more intuitive way to load and view multi-dimensional geospatial arrays. It builds on Numpy/GDAL and we will use it to load in our geospatial data.
- [Matplotlib](https://matplotlib.org/index.html) is a low-level library for creating two-dimensional diagrams and graphs. With its help, you can build diverse charts, from histograms and scatterplots to non-Cartesian coordinates graphs. Moreover, many popular plotting libraries are designed to work in conjunction with matplotlib.
- [SciPy](https://www.scipy.org/about.html) is a library that provides functions for numerical integration, interpolation, optimization, linear algebra and statistics.

**Our first step is to import them:**

In [ ]:
%%capture
import re
from pathlib import Path
import json # for loads
import math # for ceil
from datetime import datetime

import pandas as pd # for DatetimeIndex
from osgeo import gdal # for Info
gdal.UseExceptions()
import numpy as np # for copy, isnan, log10, ma.masked_where, max, mean, min, percentile, power, unique, var, where 
import scipy.signal
import xarray as xr

%matplotlib inline
import matplotlib.pylab as plb # for figure, grid, rcParams, savefig
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib import rc

from ipyfilechooser import FileChooser

from IPython.display import HTML

import opensarlab_lib as asfn
asfn.jupytertheme_matplotlib_format()

---
## 1. Load Your Prepared Data Stack Into the Notebook

**To download a set of data you can use to run this notebook, we recommend working through the [`search_download_OPERA-RTC-S1.ipynb`](../data_access/search_download_OPERA-RTC-S1.ipynb) notebook found in `data_access` folder in this repository!**

You can however use any set of OPERA RTCs as long as they have the same footprint (e.g., the same burst ID).

**Select the directory holding your tiffs**
- Click the `Select` button
- Navigate to your data directory
- Click the `Select` button
- Confirm that the desired path appears in green text
- Click the `Change` button to alter your selection

In [ ]:
fc = FileChooser(Path.cwd())
display(fc)

**Determine the path to the analysis directory containing the tiff directory:**

In [ ]:
analysis_dir = Path(fc.selected_path)
paths = list(analysis_dir.glob('*/*VV.tif')) + list(analysis_dir.glob('*/*VH.tif'))

**Load the data using Xarray**

In [ ]:
# This function is used to correctly set the date and band properties of the data when using xr.open_mfdataset
def add_time_and_pol(ds):
    name = Path(ds.encoding['source']).with_suffix('').name
    time = pd.to_datetime(name.split('_')[4])
    polarization = name.split('_')[-1].upper()
    ds = ds.assign_coords(time=time)
    ds = ds.expand_dims(dim={'time':1})
    ds.coords['band'] = [polarization]
    return ds

rtc = xr.open_mfdataset(paths, engine='rasterio', preprocess=add_time_and_pol).to_dataarray().squeeze()
rtc

---
## 3. Now You Can Work With Your Data

Now you are ready to perform time series analysis on your data stack

---
## 3.1 Open Your Data Stack and Visualize Some Layers

**Plot images and histograms for VV and VH polarizations:**

Note: Depending the histograms plotted by this cell, you may wish to adjust vmax when calling imshow() on ax1 and ax3. Increase the vmax value if the histogram cuts off much of the end of the peak, making your image too bright to see features well. Decrease vmax if the histogram extends much beyond the end of the peak, which will make your image appear dark.

In [ ]:
# Setup the pyplot plots
fig = plb.figure(figsize=(18,10)) # Initialize figure with a size
ax1 = fig.add_subplot(221)  # 221 determines: 2 rows, 2 plots, first plot
ax2 = fig.add_subplot(222)  # 222 determines: 2 rows, 2 plots, second plot
ax3 = fig.add_subplot(223)  # 223 determines: 2 rows, 2 plots, third plot
ax4 = fig.add_subplot(224)  # 224 determines: 2 rows, 2 plots, fourth plot

# Set vmin and vmax
vmin=0
vmax=0.2

# Access the data
time = rtc.time[2].values
time_name = pd.to_datetime(str(time)).strftime('%Y-%m-%d')

vv_array = rtc.sel(time=time, band='VV').squeeze().to_numpy()
vh_array = rtc.sel(time=time, band='VH').squeeze().to_numpy()

# Plot the VV image
ax1.imshow(vv_array, cmap='gray', vmin=vmin, vmax=vmax) # see note above regarding vmax adjustments
ax1.set_title(f'Image VV {time_name}')

# Flatten the VV image into a 1 dimensional vector and plot the histogram:
h = ax2.hist(vv_array.flatten(), bins=200, range=(vmin, vmax))
ax2.xaxis.set_label_text('Amplitude? (Uncalibrated DN Values)')
ax2.set_title(f'Histogram VV {time_name}')

# Plot the VH image
ax3.imshow(vh_array,cmap='gray', vmin=vmin, vmax=vmax) # see note above regarding vmax adjustments
ax3.set_title(f'Image VH {time_name}')

# Flatten the VH image into a 1 dimensional vector and plot the histogram:
h = ax4.hist(vh_array.flatten(), bins=200, range=(vmin,vmax))
ax4.xaxis.set_label_text('Amplitude? (Uncalibrated DN Values)')
ax4.set_title(f'Histogram VH {time_name}')

---
### 3.2 Calibration and Data Conversion between dB and Power Scales

**Note, that if your data were generated by HyP3, this step is not necessary!** HyP3 performs the full data calibration and provides you with calibrated data in power scale.
    
If, your data is from a different source, however, calibration may be necessary to ensure that image gray values correspond to proper radar cross section information. 

Calibration coefficients for SAR data are often defined in the decibel (dB) scale due to the high dynamic range of the imaging system. For the L-band ALOS PALSAR data at hand, the conversion from uncalibrated DN values to calibrated radar cross section values in dB scale is performed by applying a standard **calibration factor of -83 dB**. 

$\gamma^0_{dB} = 20 \cdot log10(DN) -83$

The data at hand are radiometrically terrain corrected images, which are often expressed as terrain flattened $\gamma^0$ backscattering coefficients. For forest and land cover monitoring applications $\gamma^o$ is the preferred metric.

**To apply the calibration constant for your data and export in *dB* scale, uncomment the following code cell:**

In [ ]:
# caldB = 20*np.log10(rtc)-83

While **dB**-scaled images are often "visually pleasing", they are often not a good basis for mathematical operations on data. For instance, when we compute the mean of observations, it makes a difference whether we do that in power or dB scale. Since dB scale is a logarithmic scale, we cannot simply average data in that scale. 
    
Please note that the **correct scale** in which operations need to be performed **is the power scale.** This is critical, e.g. when speckle filters are applied, spatial operations like block averaging are performed, or time series are analyzed.

To **convert from dB to power**, apply: $\gamma^o_{pwr} = 10^{\frac{\gamma^o_{dB}}{10}}$

In [ ]:
#calPwr = np.power(10,caldB/10)

---
### 3.3 Create a Time Series Animation

Now we are ready to create a time series animation from the calibrated SAR data.

**Create and move into a directory in which to store our plots and animations:**

In [ ]:
product_path = analysis_dir/f'plots_and_animations'
if not product_path.exists():
    product_path.mkdir()

**Create a masked raster stack:**

In [ ]:
vv = rtc.sel(band='VV').squeeze().values

**Get dates of stack:**

In [ ]:
dates = [pd.to_datetime(time).strftime('%Y-%m-%d') for time in rtc.time.values]

**Generate a matplotlib time-series animation:**

In [ ]:
%%capture

fig = plt.figure(figsize=(14, 8))
ax = fig.subplots()
ax.axis('off')
vmin = np.nanpercentile(vv.flatten(), 5)
vmax = np.nanpercentile(vv.flatten(), 95)

r0dB = 20 * np.log10(vv) - 83

im = ax.imshow(vv[0], cmap='gray', vmin=vmin, vmax=vmax)
ax.set_title(f'{dates[0]}')

def animate(i):
    ax.set_title(f'{dates[i]}')
    im.set_data(vv[i])

# Interval is given in milliseconds
ani = animation.FuncAnimation(fig, animate, frames=vv.shape[0], interval=400)

**Configure matplotlib's RC settings for the animation:**

In [ ]:
rc('animation', embed_limit=40971520.0)  # We need to increase the limit maybe to show the entire animation

**Create a javascript animation of the time-series running inline in the notebook:**

In [ ]:
HTML(ani.to_jshtml())

**Delete the dummy png** that was saved to the current working directory while generating the javascript animation in the last code cell.

In [ ]:
try:
    product_path/Path('None0000000.png').unlink()
except FileNotFoundError:
    pass

**Save the animation (animation.gif):**

In [ ]:
ani.save(f'{product_path}/animation.gif', writer='pillow', fps=2)

---
### 3.4 Plot the Time Series of Means Calculated Across the Subset

To create the time series of means, we will go through the following steps:
1. Ensure that you use the data in **power scale** ($\gamma^o_{pwr}$) for your mean calculations.
1. compute means.
1. convert the resulting mean values into dB scale for visualization.
1. plot time series of means.

**Compute the means:**

In [ ]:
input_data = rtc.sel(band='VV').squeeze().values
rs_means_pwr = np.nanmean(input_data, axis=(1, 2))

**Convert resulting mean value time-series to dB scale for visualization:**

In [ ]:
rs_means_dB = 10.*np.log10(rs_means_pwr)

**Plot and save the time series of means (RCSoverTime.png):**

In [ ]:
try:
    plt.rcParams.update({'font.size': 14})
    fig = plt.figure(figsize=(16, 4))
    ax1 = fig.subplots()
    window_length = len(rs_means_pwr)-1
    if window_length % 2 == 0:
        window_length -= 1
    polyorder = math.ceil(window_length*0.1)
    yhat = scipy.signal.savgol_filter(rs_means_pwr, window_length, polyorder) 
    ax1.plot(dates, yhat, color='red', marker='o', markerfacecolor='white', linewidth=3, markersize=6)
    ax1.plot(dates, rs_means_pwr, color='black', linewidth=1)
    plt.grid()
    ax1.set_xlabel('Date')
    ax1.tick_params(axis='x', labelrotation=45)
    ax1.set_ylabel(r'$\overline{\gamma^o}$ [power]')
    plt.savefig(f'{product_path}/RCSoverTime.png', dpi=72, transparent='true')
except ValueError as e:
    print(f'Error: polyorder: {polyorder} >= window_length: {window_length}')
    raise

---
### 3.6 Calculate Coefficient of Variance

The coefficient of variance describes how much the $\sigma_{0}$ or $\gamma_{0}$ measurements in a pixel vary over time. Hence, the coefficient of variance can indicate different vegetation cover and soil moisture regimes in your area.

**Write a function to convert our plots into GeoTiffs:**

In [ ]:
def geotiff_from_plot(source_image, out_filename, extent, utm, cmap=None, vmin=None, vmax=None, interpolation=None, dpi=300):
    assert "." not in out_filename, 'Error: Do not include the file extension in out_filename'
    assert type(extent) == list and len(extent) == 2 and len(extent[0]) == 2 and len(
        extent[1]) == 2, 'Error: extent must be a list in the form [[upper_left_x, upper_left_y], [lower_right_x, lower_right_y]]'
    
    plt.figure()
    plt.axis('off')
    plt.imshow(source_image, cmap=cmap, vmin=vmin, vmax=vmax, interpolation=interpolation)
    temp = f"{out_filename}_temp.png"
    plt.savefig(temp, dpi=dpi, transparent='true', bbox_inches='tight', pad_inches=0)

    cmd = f"gdal_translate -of Gtiff -a_ullr {extent[0][0]} {extent[0][1]} {extent[1][0]} {extent[1][1]} -a_srs EPSG:{utm} {temp} {out_filename}.tiff"
    !{cmd}
    try:
        Path(temp).unlink()
    except FileNotFoundError:
        pass

**Plot the Coefficient of Variance Map and save it as a png (Coeffvar.png):**

In [ ]:
input_data = rtc.sel(band='VV').squeeze().values
input_data[np.isnan(input_data)] = 0

test = np.var(input_data, axis=0) ** 0.5
mtest = np.mean(input_data, axis=0)
coeffvar = test/(mtest+0.001)

plt.rcParams.update({'font.size': 14})
fig = plt.figure(figsize=(13, 10))
ax = fig.subplots()
ax.axis('off')
vmin = np.percentile(coeffvar.flatten(), 1)
vmax = np.percentile(coeffvar.flatten(), 99)
ax.set_title('Coefficient of Variance Map')

im = ax.imshow(coeffvar, cmap='jet', vmin=vmin, vmax=vmax)
fig.colorbar(im, ax=ax, shrink=0.5)
plt.savefig(f"{product_path}/Coeffvar.png", dpi=300, transparent='true')

**Save the coefficient of variance map as a GeoTiff (Coeffvar.tiff):**

In [ ]:
from pyproj import CRS

crs = CRS.from_wkt(rtc.coords['spatial_ref'].attrs['crs_wkt'])
utm = crs.to_epsg()

minx, x_size, _, maxy, _, y_size = [float(x) for x in rtc.coords['spatial_ref'].attrs['GeoTransform'].split(' ')]
_, _, n_y, n_x = rtc.shape
coords = [(minx, maxy), (minx + (x_size * n_x), maxy + (y_size * n_y))]

In [ ]:
%%capture
geotiff_from_plot(coeffvar, f"{product_path}/Coeffvar", coords, utm, cmap='jet', vmin=vmin, vmax=vmax)

---
### 3.7 Threshold Coefficient of Variance Map

This is an example how to threshold the derived coefficient of variance map. This can be useful, e.g., to detect areas of active agriculture.

**Plot and save the coefficient of variance histogram and CDF (thresh_coeff_var_histogram.png):**

In [ ]:
plt.rcParams.update({'font.size': 14})
fig = plt.figure(figsize=(14, 6)) # Initialize figure with a size
ax1 = fig.add_subplot(121)  # 121 determines: 2 rows, 2 plots, first plot
ax2 = fig.add_subplot(122)
# Second plot: Histogram
# IMPORTANT: To get a histogram, we first need to *flatten* 
# the two-dimensional image into a one-dimensional vector.
h = ax1.hist(coeffvar.flatten(), bins=200, range=(1e-6, 1.5))
ax1.xaxis.set_label_text('Coefficient of Variation')
ax1.set_title('Coeffvar Histogram')
plt.grid()
n, bins, patches = ax2.hist(coeffvar.flatten(), bins=200, range=(1e-6, 1.5), cumulative='True', density='True', histtype='step', label='Empirical')
ax2.xaxis.set_label_text('Coefficient of Variation')
ax2.set_title('Coeffvar CDF')
plt.grid()
plt.savefig(f"{product_path}/thresh_coeff_var_histogram.png", dpi=72, transparent='true')

**Plot the Threshold Coefficient of Variance Map and save it as a png (Coeffvarthresh.png):**

In [ ]:
plt.rcParams.update({'font.size': 14})

outind = np.where(n > 0.90)
threshind = np.min(outind)
thresh = bins[threshind]
coeffvarthresh = np.copy(coeffvar)
coeffvarthresh[coeffvarthresh < thresh] = 0
coeffvarthresh[coeffvarthresh >= thresh] = 1
fig = plt.figure(figsize=(13, 10))
ax = fig.subplots()
ax.axis('off')
ax.set_title(r'Thresholded Coeffvar Map $\alpha=90\%$')
im = ax.imshow(coeffvarthresh, cmap='Blues', vmin=0, vmax=1, interpolation='none')
bar = fig.colorbar(im, ax=ax, shrink=0.5)
plt.savefig(f"{product_path}/Coeffvarthresh.png", dpi=300, transparent='true')

**Save the Threshold Coefficient of Variance Map as a GeoTiff (Coeffvarthresh.tiff):**

In [ ]:
%%capture
geotiff_from_plot(coeffvarthresh, f"{product_path}/Coeffvarthresh", coords, utm, cmap='jet', vmin=vmin, vmax=vmax)